In [3]:
import scipy
import matplotlib.pyplot as plt
import numpy
import pandas


In [4]:
column_names = ['ASIC', 'Channel', 'TimeStamp', 'Adc', 'PileUp', 'OverFlow',
                'Gain', 'Thr', 'iCal', 'Trim', 'TS Asic', 'TS Fpga']
data_calib = pandas.read_csv('raw_data_ical_no_calib.dat', delimiter='\t', names=column_names, skiprows=2)
column_names = ['ASIC', 'Channel', 'TimeStamp', 'Adc', 'OverFlow', 'PileUp']
data = pandas.read_csv('GEM_HV_3930_2_AGH_plain_part_008.pcap_raw_hits.txt', delimiter='\t', names=column_names, skiprows=1, index_col=False)


In [5]:
grouped_data = {}
slope_data = {}

def ele_calib(xs, ys):
    result = scipy.stats.linregress(xs, ys)
    return abs(result.slope), result.intercept

for (asic, channel), group in data_calib.groupby(['ASIC', 'Channel']):
    xs = group['iCal'].tolist()
    ys = group['Adc'].tolist()
    grouped_data[(asic, channel)] = (xs, ys)

for (asic, channel) in grouped_data:
    xs, ys = grouped_data[(asic, channel)]
    slope, intercept = ele_calib(xs, ys)
    slope_data[(asic, channel)] = (slope, intercept)


def calibrate_row(row):
    slope, intercept = slope_data[(row['ASIC'], row['Channel'])]
    return (row['Adc'] - intercept) / slope

In [6]:
data['Adc'] = data.apply(calibrate_row, axis=1)


In [7]:
data = data.sort_values(by='TimeStamp').reset_index(drop=True)
time_diff = data['TimeStamp'].diff().fillna(0)
new_group = (time_diff >= 100).cumsum()
data['GroupID'] = new_group
print(data['GroupID'].head(100))

0      0
1      0
2      1
3      2
4      2
      ..
95    26
96    26
97    27
98    27
99    27
Name: GroupID, Length: 100, dtype: int32


In [8]:
def transf_cords(row):
    asic = row['ASIC'] - 86
    channel = row['Channel']
    axis = 'x'
    coord = 0
    if asic > 3:
        axis = 'y'
    if axis == 'x':
        coord = 31 - channel
        coord = coord + asic * 32
    else:
        coord = channel
        asic = 3 - asic
        coord = coord + asic * 32
    return axis, coord

In [10]:
data[['axis', 'coord']] = data.apply(transf_cords, axis=1, result_type='expand')

In [ ]:
event_map = pandas.DataFrame(columns=['x', 'y', 'Value'])

for groupID, group in data.groupby(['GroupID']):
    axis = group['axis'].tolist()
    coord = group['coord'].tolist()
    values = group['Adc'].tolist()

    x_coords = [coord[i] for i in range(len(axis)) if axis[i] == 'x']
    x_values = [values[i] for i in range(len(axis)) if axis[i] == 'x']
    
    y_coords = [coord[i] for i in range(len(axis)) if axis[i] == 'y']
    y_values = [values[i] for i in range(len(axis)) if axis[i] == 'y']

    # Compute center of mass if values are present
    x_com = round(sum(c * v for c, v in zip(x_coords, x_values)) / sum(x_values)) if x_values else None
    y_com = round(sum(c * v for c, v in zip(y_coords, y_values)) / sum(y_values)) if y_values else None

    total_value = sum(values)

    new_row = pandas.DataFrame([{'x': x_com, 'y': y_com, 'Value': total_value}])
    event_map = pandas.concat([event_map, new_row], ignore_index=True)

C:\Users\Dennisov\AppData\Local\Temp\ipykernel_11728\2940396317.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  event_map = pandas.concat([event_map, new_row], ignore_index=True)


In [ ]:
print(event_map.head(10))
x_target = 10
y_target = 10
selected = event_map[(event_map['x'] == x_target) & (event_map['y'] == y_target)]
plt.figure(figsize=(8, 5))
plt.hist(selected['Value'], bins=30, color='skyblue', edgecolor='black')
plt.title(f"Histogram of Values near (x={x_target}, y={y_target})")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()